In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#!pip install imdbpy
import imdb
from imdb import Cinemagoer

In [105]:
dataset = pd.read_csv('new_dataset.csv')

In [106]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1641 entries, 0 to 1640
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            1641 non-null   object 
 1   Rating           1641 non-null   float64
 2   Year             1641 non-null   int64  
 3   Month            1641 non-null   object 
 4   Runtime          1641 non-null   float64
 5   Directors        1641 non-null   object 
 6   Budget           1641 non-null   float64
 7   Income           1641 non-null   float64
 8   Action           1641 non-null   int64  
 9   Adventure        1641 non-null   int64  
 10  Animation        1641 non-null   int64  
 11  Biography        1641 non-null   int64  
 12  Comedy           1641 non-null   int64  
 13  Crime            1641 non-null   int64  
 14  Drama            1641 non-null   int64  
 15  Family           1641 non-null   int64  
 16  Fantasy          1641 non-null   int64  
 17  History       

In [107]:
ia = imdb.IMDb()

In [113]:
# Function to find number of movies and average rating for person


def update_columns(column_name, dataset, person_type):
    # Create an IMDb object
    ia = imdb.IMDb()

    for index, row in dataset.iterrows():
        name = row[column_name]
        name_search = ia.search_person(name)
        
        # Check if name has a match on IMDb
        if not name_search:
            dataset.loc[index, f'{column_name}_Nr_Movies'] = 0
            dataset.loc[index, f'{column_name}_Avg_Rating'] = 0
            dataset.loc[index, f'{column_name}_Nr_Awards'] = 0
            dataset.loc[index, f'{column_name}_Nr_Nominations'] = 0
            continue

        person_id = name_search[0].getID()
        person = ia.get_person(person_id)
        ia.update(person, 'awards')        

        # Extract the number of awards and nominees
        awards_dict = person['awards']
        num_awards = 0
        num_nominations = 0

        for award in awards_dict:
            if 'wins' in award:
                num_awards += award['wins']
            if 'nominations' in award:
                num_nominations += award['nominations']

        # Set the number of awards and nominations to 0 if not found
        if num_awards == 0:
            dataset.loc[index, f'{column_name}_Nr_Awards'] = 0
        if num_nominations == 0:
            dataset.loc[index, f'{column_name}_Nr_Nominations'] = 0

        # Extract the movies and ratings for the actor
        ia.update(person, 'filmography')
        ratings_list = []

        # Iterate over the actor's filmography and extract the movie id
        for movie in person['filmography'][person_type]:
            movie_title = movie.movieID

            # Get the movie's rating based on the movie id
            movie = ia.get_movie(movie_title)
            release_year = movie.get('year')

            # Handle release years
            if release_year is not None:
                release_year = int(release_year)
            else:
                release_year = 0

            # Keep only ratings with release year up to 2023
            rating = movie.get('rating')
            if 0 < release_year < 2024 and rating is not None:
                ratings_list.append(float(rating))

        # Set the number of movies and average rating to 0 if not found
        if not ratings_list:
            dataset.loc[index, f'{column_name}_Nr_Movies'] = 0
            dataset.loc[index, f'{column_name}_Avg_Rating'] = 0
        else:
            dataset.loc[index, f'{column_name}_Nr_Movies'] = len(ratings_list)
            dataset.loc[index, f'{column_name}_Avg_Rating'] = np.mean(ratings_list)

        print(index)

    return dataset


## 1. Finding the number of movies and average rating for Director

In [114]:
# Running the function above
update_columns('Directors', dataset, 'director')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


KeyError: 'director'

In [115]:
dataset.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1641 entries, 0 to 1640
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            1641 non-null   object 
 1   Rating           1641 non-null   float64
 2   Year             1641 non-null   int64  
 3   Month            1641 non-null   object 
 4   Runtime          1641 non-null   float64
 5   Directors        1641 non-null   object 
 6   Budget           1641 non-null   float64
 7   Income           1641 non-null   float64
 8   Action           1641 non-null   int64  
 9   Adventure        1641 non-null   int64  
 10  Animation        1641 non-null   int64  
 11  Biography        1641 non-null   int64  
 12  Comedy           1641 non-null   int64  
 13  Crime            1641 non-null   int64  
 14  Drama            1641 non-null   int64  
 15  Family           1641 non-null   int64  
 16  Fantasy          1641 non-null   int64  
 17  History       

## 2. Finding the number of movies and average rating for Lead stars

In [116]:
# Running the function above
update_columns('Lead', dataset, 'actor')

0
1
2


KeyError: 'awards'

## 3. Finding the number of movies and average rating for Supporting stars

In [ ]:
# Running the function above
update_columns('Supporting', dataset, 'actor')

KeyError: 'actor'

In [ ]:
dataset.info()